# Crystallinity maps of semicrystalline polymer (R-BAPB) 
## 2D X-ray scattering experiment



In [ ]:
import h5py
import numpy as np
import scipy as scipy
from scipy import optimize
from scipy import signal
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
import matplotlib.ticker as ticker
import time
import peakutils

%matplotlib inline

In [ ]:
#Initial global parameters

file_name = "H://LP/R-BAPB/print72.h5"
file_num = '72'
data_inner_directory = "data/data" # path to data inside the file
map_width = 1 #default value
min_ROI_q = 570 # region of interest for q, data point indices
max_ROI_q = 650
min_ROI_chi = 0 # region of interest for angle chi, data point indices
max_ROI_chi = 720 

In [ ]:
#Opening h5 file
file = h5py.File(file_name, "r") # reading mode
all_scans = file[data_inner_directory] 
map_width = int(all_scans.shape[0]**0.5) 
q_set = file["data/q"] # actual values for q and chi


chi_set = file["data/chi"]

In [ ]:
np.save('q_set',q_set)
np.save('chi_set', chi_set)

In [ ]:
scan = all_scans[1000]
plt.imshow(scan, cmap = 'jet', clim = (0.01,1))

In [ ]:
plt.hist(scan[1], bins = 300)
plt.show()

In [ ]:
scan = np.load('scan_sum1434.npy')

In [ ]:
for row in scan:
    row[row<0]= np.nan
    
print (scan[10])

In [ ]:
#Global normalization
mu = np.nanmean(scan)
sigma = np.nanstd(scan)

mu,sigma

In [ ]:
for row in scan:
    for cell in row:
        if cell:
            cell = (cell-mu)/sigma

In [ ]:
plt.imshow(scan, cmap = 'jet', clim = (0.01,0.7))

In [ ]:

min_ROI_q = 570 # region of interest for q, data point indices
max_ROI_q = 650

profile_sum = np.zeros(all_scans.shape[2], dtype=np.float32)
        
j_counter = 0
for j in range (min_ROI_chi,max_ROI_chi):
    profile = np.nan_to_num(scan[j])           
    profile_sum += profile
    j_counter+=1
#profile_sum/=j_counter


x = q_set[min_ROI_q:max_ROI_q]
y = profile_sum[min_ROI_q:max_ROI_q]

plt.figure(figsize=(10,6))
plt.plot(x,y)
profile_sum

In [ ]:
#local normalisation
local_mu = np.nanmean(profile_sum[min_ROI_q-100:max_ROI_q+100])
local_sigma = np.nanstd(profile_sum[min_ROI_q-100:max_ROI_q+100])

local_profile = profile_sum[min_ROI_q-100:max_ROI_q+100]

for cell in local_profile:
    cell = (cell - local_mu)/local_sigma

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(q_set[min_ROI_q:max_ROI_q],local_profile[100:-100])


In [ ]:
local_profile[100:-100] == profile_sum[min_ROI_q:max_ROI_q]

In [ ]:
#without normalization
new_scan = np.load('scan_sum1434.npy')

min_ROI_q = 570 # region of interest for q, data point indices
max_ROI_q = 650

new_profile_sum = np.zeros(all_scans.shape[2], dtype=np.float32)
        
j_counter = 0
for j in range (min_ROI_chi,max_ROI_chi):
    profile = new_scan[j]    
    if (-10 not in profile[min_ROI_q:max_ROI_q]):
        new_profile_sum += profile
        j_counter+=1
#profile_sum/=j_counter


x = q_set[min_ROI_q:max_ROI_q]
y = new_profile_sum[min_ROI_q:max_ROI_q]

plt.figure(figsize=(10,6))
plt.plot(x,y)
new_profile_sum

In [ ]:
#Mapping by simply integrating in ROI_q


#baseline = np.load('1434profile.npy')
start_time = time.time() #time counter
stn_map_data = np.zeros((map_width,map_width), dtype=np.float32) 

for i in range(map_width**2):
        current_scan = all_scans[i]   
        profile_sum = np.zeros(all_scans.shape[2], dtype=np.float32)
        
        j_counter = 0
        for j in range (min_ROI_chi,max_ROI_chi):
            profile = current_scan[j]           
            if (-10 not in profile[min_ROI_q-70:max_ROI_q+70]):
                profile_sum += profile
                j_counter+=1
        profile_sum/=j_counter
        
        
        y_wide = scipy.signal.savgol_filter(profile_sum[min_ROI_q-70:max_ROI_q+70], window_length= 25, polyorder= 3, deriv=0, delta=0.1, axis=-1, mode='interp', cval=0.0)
        #base = peakutils.baseline(profile_sum[min_ROI_q-70:max_ROI_q+70], deg=5)
        #y_wide= profile_sum[min_ROI_q-70:max_ROI_q+70]-base
        

        x_set = q_set[min_ROI_q:max_ROI_q]
        y_set = y_wide[70:-70]

        #popt, pcov = scipy.optimize.curve_fit(gaussian, x, y, p0=[1.3,0.05,0.5,0.1], method = 'dogbox')
        I = scipy.integrate.trapz(x = x_set, y = y_set)
        print(I)
        
        
        x = int(i%map_width)
        y = int(i//map_width)
        
        stn_map_data[y, x]=I
        print(i,"--- %s seconds ---" % (time.time() - start_time))

#saving map data

np.save(file_num + 'trapz_map_data', stn_map_data)

In [ ]:
#Summing all  scans of the experiment


scan_sum = np.zeros((all_scans.shape[1],all_scans.shape[2]), dtype = np.float32)
start_time = time.time()
for scan in all_scans:
    scan_sum+=scan
    print("--- %s seconds ---" % (time.time() - start_time))
    
scan_sum/=all_scans.shape[0] #normalizing

#saving

np.save('scan_sum' + file_num, scan_sum)

In [ ]:
#Assembling a normilized profile from different parts of a scan (because yeah I don't have mask files)

#scan_sum = np.load('scan_sum' + file_num + '.npy', )

#TODO - зашкал при i=6 - почему???
#Сшивка между срезами - чем больше N, тем меньше разница

M = all_scans.shape[2] #number of points in profile
N=3 #number of slices
halo_profile = np.zeros(M, dtype=np.float32)
for i in range(N):  
    
    min_slice = int(M*i/N)
    max_slice = int(M*(i+1)/N)
    counter = 0 
    print(i, min_slice, max_slice)
    local_sum = np.zeros(M, dtype=np.float32)
    
    for j in range(min_ROI_chi, max_ROI_chi):
        profile = scan_sum[j]
        if not any(x<=0 for x in profile[min_slice:max_slice]):
            local_sum[min_slice:max_slice]+=profile[min_slice:max_slice]        
            counter +=1
                
    print(counter)
    halo_profile[min_slice:max_slice] += (local_sum[min_slice:max_slice]/counter) 


fig = plt.figure(figsize=(15,10))
plt.plot(q_set,halo_profile, label = 'halo profile')


#Setting scale 
ax = fig.add_subplot(2, 1, 1)
ax.loglog(q_set,halo_profile, label = 'loglog halo profile')

#ax.set_xscale('log')
#ax.set_yscale('log')
plt.legend()

#saving
np.save('halo_profile' + file_num, scan_sum)
np.savetxt('halo_profile' + file_num + '.txt', halo_profile)
np.savetxt('q_set' + file_num + '.txt', q_set)


In [ ]:
#Loading corrected halo profile
halo_baseline = np.loadtxt('halo_baseline' + file_num +'.txt')
print(halo_baseline)
#Integrating, saving

#1434 halo slice: 45-1998
halo_min = 1
halo_max = 2000

I =scipy.integrate.trapz(y = halo_baseline, x = q_set[halo_min-1:halo_max])
print (I)

halo_pars = [halo_min, halo_max, I]
np.savetxt('halo_params' + file_num + '.txt', halo_pars)


In [ ]:
#Mapping by simply integrating in ROI_q

#baseline = np.load('1434profile.npy')
start_time = time.time() #time counter
simple_map_data = np.zeros((map_width,map_width), dtype=np.float32) 

for i in range(map_width**2):
        current_scan = all_scans[i]   
        profile_sum = np.zeros(all_scans.shape[2], dtype=np.float32)
        
        j_counter = 0
        for j in range (min_ROI_chi,max_ROI_chi):
            profile = current_scan[j]           
            if (-10 not in profile[min_ROI_q-70:max_ROI_q+70]):
                profile_sum += profile
                j_counter+=1
        profile_sum/=j_counter
        
        base = peakutils.baseline(profile_sum[min_ROI_q-70:max_ROI_q+70], deg=5)
        profile_sum[min_ROI_q-70:max_ROI_q+70]-=base
        I =scipy.integrate.trapz(y = profile_sum[min_ROI_q:max_ROI_q], x = q_set[min_ROI_q:max_ROI_q])
        x = int(i%map_width)
        y = int(i//map_width)
        
        simple_map_data[y, x]=I
        print(i,"--- %s seconds ---" % (time.time() - start_time))

#saving map data

#np.save(file_num + 'simple_map_data', simple_map_data)

In [ ]:
plt.imshow(simple_map_data, cmap = "jet") #jet, magma, inferno, copper
plt.figure(figsize=(15,15))

plt.imsave(file_num +"map_q_{}-{}_chi_{}-{}-savitskiy_jet.png".format(min_ROI_q, max_ROI_q, min_ROI_chi, max_ROI_chi), \
          simple_map_data, cmap = "jet")


In [ ]:
print(simple_map_data)

In [ ]:
#Creating histogram
test_data = simple_map_data.flatten()
plt.hist(test_data, bins = 300)
plt.show()
#saving

In [ ]:
#Thresholding map data after looking at a histogram

masked_simple_map_data = np.zeros((map_width,map_width), dtype=np.float32) 
for i in range(map_width**2):
    x = int(i%map_width)
    y = int(i//map_width)
    if simple_map_data[x,y]>22:
        masked_simple_map_data[x,y] = simple_map_data[x,y] 
        
plt.imshow(masked_simple_map_data, cmap = "jet")
plt.figure(figsize=(15,15))

In [ ]:
#Creating mask

mask = np.load('1434mask_data.npy')
plt.imshow(mask, cmap = "jet") #jet, magma, inferno, copper
plt.figure(figsize=(15,15))
#mask = masked_simple_map_data > 0
#saving map data and mask

#plt.imsave(file_num +"map_q_{}-{}_chi_{}-{}-masked_jet.png".format(min_ROI_q, max_ROI_q, min_ROI_chi,\
 #                                                                  max_ROI_chi), masked_simple_map_data, cmap = "jet")
#np.save(file_num + "masked_map_data", masked_simple_map_data)
#np.save(file_num + "mask_data", mask)

In [ ]:
#Getting a normalized masked  profile of ROI_q
scan_count = 0
masked_scan_sum = np.zeros((all_scans.shape[1],all_scans.shape[2]), dtype = np.float32)
start_time = time.time()
for i in range(map_width**2):
    x = int(i%map_width)
    y = int(i//map_width)
    if not mask[y,x]:
        masked_scan_sum+=all_scans[i]
        scan_count +=1
        
    print(i,"--- %s seconds ---" % (time.time() - start_time))
    
masked_scan_sum/=scan_count #normalizing

#saving
np.save(file_num + 'negative_masked_scan_sum', masked_scan_sum)

In [ ]:
plt.imshow(map_data, cmap = "magma")
plt.figure(figsize=(15,15))

In [ ]:
file.close()